<a href="https://colab.research.google.com/github/david-meltzer/LLMs/blob/main/sft_llama_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dependecies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/SFT

/content/drive/MyDrive/SFT


In [ ]:
%ls

artifacts/         requirements.txt   utils/
finetuned_models/  sft_llama-2.ipynb  wandb/


In [ ]:
!pip install -qqqU -r requirements.txt
!pip install -qqqU datasets trl

In [ ]:
import torch, datasets, accelerate, transformers, evaluate, gc
import bitsandbytes as bnb

from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          GenerationConfig,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          BitsAndBytesConfig)

from peft import (LoraConfig,
                  get_peft_model,
                  prepare_model_for_kbit_training)

from trl import (SFTTrainer,
                 DataCollatorForCompletionOnlyLM)

from google.colab import runtime


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [ ]:
# !python -c "import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'"
# !pip install -qqqU ninja packaging
# !MAX_JOBS=4 pip install flash-attn -qqqU --no-build-isolation

# use_flash_attention = False

# if torch.cuda.get_device_capability()[0] >= 8:
#     from utils.llama_patch import replace_attn_with_flash_attn
#     print("Using flash attention")
#     replace_attn_with_flash_attn()
#     use_flash_attention = True

In [ ]:
from pprint import pprint

def pp(x):
  pprint(x, compact = True, width = 150)

def cleanup(x):
    if x == 'memory':
        gc.collect()
        torch.cuda.empty_cache()
    elif x == 'runtime':
        runtime.unassign()

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

#Dataset

In [ ]:
wandb_api = wandb.Api()
artifact = wandb_api.artifact('ft-llmmm/ELI5_analysis/combined_dataset:v5', type='dataset')
artifact_dir = artifact.download()

In [ ]:
artifact_dir = './artifacts/combined_dataset:v5'

In [ ]:
from datasets import load_from_disk

data = load_from_disk(artifact_dir)
data

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'source', 'QA'],
        num_rows: 72214
    })
    validation: Dataset({
        features: ['question', 'answer', 'source', 'QA'],
        num_rows: 1964
    })
    test: Dataset({
        features: ['question', 'answer', 'source', 'QA'],
        num_rows: 3301
    })
})

In [ ]:
#Add QA number of words to the dataset

def add_num_words(example):
    example['num_words'] = len(example['QA'].split())
    return example

In [ ]:
data = data.map(add_num_words)
data = data.filter(lambda x : x['num_words'] < 400)
data

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 68587
    })
    validation: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 1890
    })
    test: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 3107
    })
})

In [ ]:
data.save_to_disk('artifacts/wiki+eli5<400')

Saving the dataset (0/1 shards):   0%|          | 0/68587 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1890 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3107 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk
data = load_from_disk('artifacts/wiki+eli5<400')

In [ ]:
wiki = data.filter(lambda example : example['source'] == 'simple_wiki')
wiki

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 29987
    })
    validation: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 1000
    })
})

In [ ]:
eli5 = data.filter(lambda example : example['source'] == 'ELI5')
eli5

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 38600
    })
    validation: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 890
    })
    test: Dataset({
        features: ['question', 'answer', 'source', 'QA', 'num_words'],
        num_rows: 2107
    })
})

#Tokenizer, formatting, collator

In [ ]:
model_id = 'meta-llama/Llama-2-7b-hf'

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# set up the collator

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['question'])):
        text = f"### Human: {example['question'][i]}\n ### Assistant: {example['answer'][i]}"
        output_texts.append(text)
    return output_texts

response_template = "### Assistant:"
collator = DataCollatorForCompletionOnlyLM(response_template=response_template, tokenizer=tokenizer)

##Word length statistics

In [ ]:
# data_sorted = data.sort('num_words')

# import numpy as np
# len_stat = np.zeros((3,400))
# split_names = {'train' : 0, 'validation' : 1 , 'test' : 2}

# for k, v in split_names.items():
#     lengths = np.array(data_sorted[k]['num_words'])
#     len_stat[v] = np.array([lengths[lengths ==n].shape[0] for n in range(400)] )

# import matplotlib.pyplot as plt

# fig, axs = plt.subplots(1,3, figsize=(10, 3))
# fig.suptitle('Number of words distibution')

# for k, v  in split_names.items():

#   axs[v].plot(len_stat[v, :])
#   axs[v].set_title(k + ' dataset')
#   if v ==0:
#     axs[v].set_xlabel('Number of words')
#     axs[v].set_ylabel('Number of examples')

# plt.tight_layout()

#Model setup

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split(".")
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if "lm_head" in lora_module_names:
        lora_module_names.remove("lm_head")
    return list(lora_module_names)

In [ ]:
def prepare_quant_model(model_id = model_id):

    nf4_config = BitsAndBytesConfig(load_in_4bit=True,
                                    bnb_4bit_quant_type="nf4",
                                    bnb_4bit_use_double_quant=True,
                                    bnb_4bit_compute_dtype=torch.bfloat16
                                    )

    model = AutoModelForCausalLM.from_pretrained(model_id,
                                                quantization_config=nf4_config,
                                                use_cache=False,
                                                device_map='auto',
                                                torch_dtype = torch.bfloat16
                                                )
    model.config.pretraining_tp = 1

    # if use_flash_attention:
    #     from utils.llama_patch import forward
    #     assert model.model.layers[0].self_attn.forward.__doc__ == forward.__doc__, "Model is not using flash attention"


    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)

    # if use_flash_attention:
    #     from utils.llama_patch import upcast_layer_for_flash_attention
    #     model = upcast_layer_for_flash_attention(model, torch.bfloat16)

    return model

def get_lora_model(model, lora_rank = 32):

    config = LoraConfig(r=lora_rank,
                        lora_alpha=32,
                        target_modules=find_all_linear_names(model),
                        lora_dropout=0.1,
                        bias="none",
                        task_type="CAUSAL_LM")

    model = get_peft_model(model, config)

    return model

#Training

In [ ]:
%env WANDB_PROJECT=sft_llama_bm
WANDB_LOG_MODEL="checkpoint"

env: WANDB_PROJECT=sft_llama_bm


In [ ]:
def training_args_config(lora_rank = 32,
                         num_epochs = 1,
                         lr = 1e-3,
                         weight_decay = 0.1,
                         batch_size = 8,
                         gradient_accumulation_steps = 4,
                         group_by_length = True,
                         eval_steps = 500,
                         find_batch_size = True,
                         lr_schedule = 'linear',
                         output_dir = 'finetuned_models/llama',
                         run_name = None
                         ):

    params = {

    #-----training----------------
    'num_train_epochs' : num_epochs,
    'learning_rate' : lr,
    'weight_decay' : weight_decay,
    'dataloader_num_workers' : 4,
    'per_device_train_batch_size' : batch_size,
    'per_device_eval_batch_size' : batch_size,
    'group_by_length' : group_by_length,
    'auto_find_batch_size' : find_batch_size,
    'lr_scheduler_type' : lr_schedule,

    #-----efficient training-----
    'gradient_accumulation_steps' : gradient_accumulation_steps,
    'gradient_checkpointing' : True,
    'optim' : 'paged_adamw_8bit',
    'bf16' : True,

    #-----logging----------------
    'report_to' : 'wandb',
    'output_dir' : output_dir,

    'evaluation_strategy' : "steps",
    'eval_steps' : eval_steps,

    'logging_strategy' : 'steps',
    'logging_steps' : eval_steps,

    'save_strategy' : "steps",
    'save_steps' : eval_steps,

    'save_total_limit' : 2,
    'load_best_model_at_end' : True,
    'metric_for_best_model' : "eval_loss",
    'greater_is_better' : False,

    'run_name' : f'r{lora_rank}_lr{lr:.0e}_wd{weight_decay}_bs{batch_size}' + ('_' + run_name if run_name is not None else ''),
    }

    params['output_dir'] = params['output_dir'] + '/' + params['run_name']

    return params

In [ ]:
r = 16
model= prepare_quant_model(model_id = model_id)

# if use_flash_attention:
#     from utils.llama_patch import forward
#     assert model.model.layers[0].self_attn.forward.__doc__ == forward.__doc__, "Model is not using flash attention"

# model = get_lora_model(model, lora_rank = r)

# if use_flash_attention:
#         from utils.llama_patch import upcast_layer_for_flash_attention
#         model = upcast_layer_for_flash_attention(model, torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tot_steps = wiki['train'].num_rows / (26*4)
tot_steps/3

96.11217948717949

In [ ]:
hyperparams = training_args_config( lora_rank = r,
                                    num_epochs = 3,
                                    lr = 1e-5,
                                    weight_decay = 0.1,
                                    batch_size = 26,
                                    gradient_accumulation_steps = 4,
                                    group_by_length = False,
                                    eval_steps = 96,
                                    find_batch_size = True,
                                    lr_schedule = 'cosine',
                                    output_dir = 'finetuned_models/llama',
                                    run_name = 'flashattn_wiki'
                                    )
pp(hyperparams)

trainer = SFTTrainer(   model,
                        args = TrainingArguments(**hyperparams),
                        max_seq_length=1024,
                        train_dataset=wiki['train'],
                        eval_dataset=wiki['validation'],
                        formatting_func=formatting_prompts_func,
                        data_collator=collator,
                        #packing = False,
                        #dataset_text_field = 'QA'
)

{'auto_find_batch_size': True,
 'bf16': True,
 'dataloader_num_workers': 4,
 'eval_steps': 96,
 'evaluation_strategy': 'steps',
 'gradient_accumulation_steps': 4,
 'gradient_checkpointing': True,
 'greater_is_better': False,
 'group_by_length': False,
 'learning_rate': 1e-05,
 'load_best_model_at_end': True,
 'logging_steps': 96,
 'logging_strategy': 'steps',
 'lr_scheduler_type': 'cosine',
 'metric_for_best_model': 'eval_loss',
 'num_train_epochs': 3,
 'optim': 'paged_adamw_8bit',
 'output_dir': 'finetuned_models/llama/r16_lr1e-05_wd0.1_bs26_flashattn_wiki',
 'per_device_eval_batch_size': 26,
 'per_device_train_batch_size': 26,
 'report_to': 'wandb',
 'run_name': 'r16_lr1e-05_wd0.1_bs26_flashattn_wiki',
 'save_steps': 96,
 'save_strategy': 'steps',
 'save_total_limit': 2,
 'weight_decay': 0.1}


Using pad_token, but it is not set yet.


In [ ]:
print_trainable_parameters(model)

trainable params: 39976960 || all params: 3540389888 || trainable%: 1.1291682911958425


In [ ]:
trainer.train()

In [ ]:
wandb.finish();

train/epoch,▁
train/global_step,▁
train/learning_rate,▁
train/loss,▁
train/epoch,0.33
train/global_step,96
train/learning_rate,1e-05
train/loss,6.8556


In [ ]:
del model
cleanup('memory')

In [ ]:
cleanup('runtime')

# Inference

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from datasets import load_from_disk
data = load_from_disk('artifacts/wiki+eli5<400')
wiki = data.filter(lambda example : example['source'] == 'simple_wiki')
eli5 = data.filter(lambda example : example['source'] == 'ELI5')

In [ ]:
base_models = ['meta-llama/Llama-2-7b-hf',
               'meta-llama/Llama-2-13b-hf']

lora_layers_trained = ['finetuned_models/llama/wiki, lr = 1e-5, wd = 0.1/checkpoint-576',
                       'finetuned_models/llama13B/r16_lr1e-05_wd0.1_bs16_wiki/checkpoint-936']

idx = 1

tokenizer = AutoTokenizer.from_pretrained(base_models[idx])
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_4bit = AutoModelForCausalLM.from_pretrained(base_models[idx], quantization_config=bnb_config)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from peft import PeftModel

model_trained = PeftModel.from_pretrained(model = model_4bit,
                    model_id = lora_layers_trained[idx],
                    torch_dtype = torch.bfloat16,
                    is_trainable = False)

#model = model_trained.merge_and_unload()

In [ ]:
def generate_examples(model, data):

    model.config.pad_token_id = model.config.eos_token_id

    generation_config = GenerationConfig(   num_beams = 4,
                                            #max_length = 100,
                                            max_new_tokens = 100,
                                            do_sample = True,
                                            temperature = 1.,
                                            top_k = 50,
                                            top_p = 0.8,
                                            repetition_penalty = 1.0,
                                            #pad_token_id = model.config.eos_token_id
                                        )




    prompts = [ f'### Human: {q}\n ### Assistant:' for q in data['question'] ]
    input = tokenizer(prompts, return_tensors = 'pt', padding = True).to(device)


    output_ids = model.generate(input_ids = input['input_ids'],
                                attention_mask = input['attention_mask'],
                                generation_config = generation_config,
                                pad_token_id = model.config.eos_token_id,
                                )

    predictions =  [tokenizer.decode(ids, skip_special_tokens = True) for ids in output_ids]

    return predictions

In [ ]:
len(wiki['test'])

1000

In [ ]:
import random
random.seed(123321)

idx = [random.randint(0,len(wiki['test'])) for i in range(10)]

In [ ]:
idx

[141, 178, 704, 255, 885, 863, 382, 270, 904, 32]

In [ ]:
batch = wiki['test'][idx]

references = batch['QA']
predictions = generate_examples(model_4bit, batch)
predictions_trained = generate_examples(model_trained, batch)

In [ ]:
for ref, pred, pred_trained in zip(references, predictions, predictions_trained):
    print(f'Reference: {ref}')
    print(50*'-')
    print(f'Prediction: {pred}')
    print(50*'-')
    print(f'SFT: {pred_trained}')
    print(150*'=')


Reference: ### Human: What is Bondi Beach known for?
 ### Assistant: Bondi Beach is a beach in Sydney, Australia, about one kilometre long and roughly seven kilometres from the centre of the city.
Bondi beach is one of the world's greatest beaches, and is one of Sydney's main tourist attractions. Large numbers of tourists visit Bondi Beach throughout the year, and many Irish and British tourists spend Christmas Day there.
--------------------------------------------------
Prediction: ### Human: What is Bondi Beach known for?
 ### Assistant: Bondi Beach is a beach and suburb in eastern Sydney, in the state of New South Wales, Australia. It is seven kilometres east of the Sydney central business district, in the local government area of Waverley Council. Bondi Beach is one of the most visited tourist sites in Australia. The beach is a popular destination for local swimmers, surfers, and sunbathers. It is also a popular backdrop for movies, television shows, and commercials.
-------------

## Rouge

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
!pip install evaluate rouge_score -qqqU
import evaluate
rouge = evaluate.load("rouge")

In [ ]:
result = rouge.compute(predictions=["The quick black tiger jumps over the lazy cat"],
                       references=["The quick brown fox walks over the lazy dog"])
print(result)

{'rouge1': 0.5555555555555556, 'rouge2': 0.375, 'rougeL': 0.5555555555555556, 'rougeLsum': 0.5555555555555556}


In [ ]:
references = []
predictions = []
predictions_trained = []

In [ ]:
for i in tqdm(range(50)):
    batch = wiki['test'][i*10 : (i+1)*10]

    references += batch['QA']
    predictions += generate_examples(model_4bit, batch)
    predictions_trained += generate_examples(model_trained, batch)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
len(predictions)

500

In [ ]:
scores_base = rouge.compute(predictions=predictions,
                            references=references)

scores_trained = rouge.compute(predictions=predictions_trained,
                               references=references)

print(f'Base model rouge scores:    {scores_base}')
print(f'Trained model rouge scores: {scores_trained}')


Base model rouge scores:    {'rouge1': 0.5073341657308709, 'rouge2': 0.31618795492913715, 'rougeL': 0.4154074302351301, 'rougeLsum': 0.42934900439508294}
Trained model rouge scores: {'rouge1': 0.5095403616375263, 'rouge2': 0.3174833679008695, 'rougeL': 0.4177316099029409, 'rougeLsum': 0.43152785924481174}


For Llama 7B there is essentially no improvement in rouge score.

Base Llama-2-7B: {'rouge1': 0.468, 'rouge2': 0.279, 'rougeL': 0.386, 'rougeLsum': 0.399}

SFT on wiki: {'rouge1': 0.470, 'rouge2': 0.279, 'rougeL': 0.385, 'rougeLsum': 0.398}

For Llama 13B there is also no improvement in rouge score.

Base Llama-2-7B: {'rouge1': 0.507, 'rouge2': 0.316, 'rougeL': 0.415, 'rougeLsum': 0.429}

SFT on wiki: {'rouge1': 0.509, 'rouge2': 0.317, 'rougeL': 0.418, 'rougeLsum': 0.431}

In [ ]:
del model_4bit
del model_trained

In [ ]:
cleanup('memory')